we could start with clean but jst for fun i am starting with dirty and cleaning the data again 

In [2]:
import pandas as pd
import numpy as np 
df = pd.read_csv("../../data/battery_capacity_dirty.csv")
df

,cycle,capacity,temperature
0,1,2.50,25.0
1,2,2.47,25.0
2,3,NaN,26.0
3,4,2.40,26.0
4,4,2.40,26.0
5,5,2.35,NaN
6,6,2.30,27.0


In [3]:
# drop row with missing capacity i am not droping the row having nan in temperature
df = df.dropna(subset=["capacity"])
df

,cycle,capacity,temperature
0,1,2.50,25.0
1,2,2.47,25.0
3,4,2.40,26.0
4,4,2.40,26.0
5,5,2.35,NaN
6,6,2.30,27.0


In [4]:
# removing duplicates there is no duplicate line so therefore none removed
df = df.drop_duplicates()
df


,cycle,capacity,temperature
0,1,2.50,25.0
1,2,2.47,25.0
3,4,2.40,26.0
5,5,2.35,NaN
6,6,2.30,27.0


In [5]:
# filling missing temperature with mean 
mean_temp = df["temperature"].mean()
df["temperature"] = df["temperature"].fillna(mean_temp)
df

/var/folders/z7/3jqbff795ld6bxk0lyv3ft2w0000gn/T/ipykernel_55738/37060144.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["temperature"] = df["temperature"].fillna(mean_temp)


,cycle,capacity,temperature
0,1,2.50,25.00
1,2,2.47,25.00
3,4,2.40,26.00
5,5,2.35,25.75
6,6,2.30,27.00


In [6]:
df.columns

Index(['cycle', 'capacity', 'temperature'], dtype='object')

now after cleaning it fast i am loading another file that was saved as clean yesterday so that it doesn't break my flow 

In [9]:
df = pd.read_csv("../../data/battery_capacity_clean.csv")
df

,cycle,capacity,temperature
0,1,2.50,25.00
1,2,2.47,25.00
2,4,2.40,26.00
3,5,2.35,25.75
4,6,2.30,27.00


In [ ]:
now i added one extra column

In [13]:
initial_capacity = df["capacity"].iloc[0]
df["capacity_fade_percent"] = ((initial_capacity - df["capacity"]) * 100)
df

,cycle,capacity,temperature,capacity_fade_percent
0,1,2.50,25.00,0.0
1,2,2.47,25.00,3.0
2,4,2.40,26.00,10.0
3,5,2.35,25.75,15.0
4,6,2.30,27.00,20.0


adding .diff() will add one column Ci - Ci-1 this will tell you trend small value gradual degradation sudden damange then large negative value

In [15]:
df["capacity_diff"] = df["capacity"].diff()
df

,cycle,capacity,temperature,capacity_fade_percent,capacity_diff
0,1,2.50,25.00,0.0,NaN
1,2,2.47,25.00,3.0,-0.03
2,4,2.40,26.00,10.0,-0.07
3,5,2.35,25.75,15.0,-0.05
4,6,2.30,27.00,20.0,-0.05


In [17]:
#now we wil add rolling mean for noise smoothing
df["rolling_capacity_mean"] = df["capacity"].rolling(window=2).mean()
df

,cycle,capacity,temperature,capacity_fade_percent,capacity_diff,rolling_capacity_mean
0,1,2.50,25.00,0.0,NaN,NaN
1,2,2.47,25.00,3.0,-0.03,2.485
2,4,2.40,26.00,10.0,-0.07,2.435
3,5,2.35,25.75,15.0,-0.05,2.375
4,6,2.30,27.00,20.0,-0.05,2.325


adding rolling capapcity will take the average of current + previous value window  = 2 means  (Ci + Ci-1)/2 this will reduce the noise

In [18]:
df["degradation_rate"] = df["capacity_fade_percent"].diff()
df

,cycle,capacity,temperature,capacity_fade_percent,capacity_diff,rolling_capacity_mean,degradation_rate
0,1,2.50,25.00,0.0,NaN,NaN,NaN
1,2,2.47,25.00,3.0,-0.03,2.485,3.0
2,4,2.40,26.00,10.0,-0.07,2.435,7.0
3,5,2.35,25.75,15.0,-0.05,2.375,5.0
4,6,2.30,27.00,20.0,-0.05,2.325,5.0


In [2]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(6,4))
plt.plot(df["cycle"], df["capacity_fade_percent"], marker = "o")
plt.xlabel("cycle")
plt.ylabel("Capacity Fade (%)")
plt.title("Capacity Fade Progression")
plt.grid(True)
plt.savefig("../../figures/degradation_rate_plot.png", dpi = 300)
plt.show()


NameError: name 'df' is not defined

<Figure size 600x400 with 0 Axes>

In [27]:
df.to_csv("../../data/battery_capacity_featured.csv", index=False)